In [3]:
import pandas as pd
from arctic import Arctic

db = Arctic("localhost")
db.initialize_library("Bitcoin")


Library created, but couldn't enable sharding: no such command: 'enablesharding', full error: {'ok': 0.0, 'errmsg': "no such command: 'enablesharding'", 'code': 59, 'codeName': 'CommandNotFound'}. This is OK if you're not 'admin'


In [5]:
df = pd.read_csv("bitcoin_data.csv")

df = pd.read_csv("bitcoin_data.csv")
df["timestamp"] = pd.to_datetime(df["Timestamp"])
df.set_index("timestamp", inplace=True)

print(df.info())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 452 entries, 2017-01-01 to 2018-03-28
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Timestamp          452 non-null    object 
 1   Open               452 non-null    float64
 2   High               452 non-null    float64
 3   Low                452 non-null    float64
 4   Close              452 non-null    float64
 5   Volume (BTC)       452 non-null    float64
 6   Volume (Currency)  452 non-null    float64
 7   Weighted Price     452 non-null    float64
dtypes: float64(7), object(1)
memory usage: 31.8+ KB
None


/tmp/ipykernel_889768/722449821.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["timestamp"] = pd.to_datetime(df["Timestamp"])


In [7]:
bitcoin_library = db["Bitcoin"]
bitcoin_library.write("BTC", df)


VersionedItem(symbol=BTC,library=arctic.Bitcoin,data=<class 'NoneType'>,version=1,metadata=None,host=localhost)

In [8]:
new_df = bitcoin_library.read("BTC").data
print(new_df)


               Timestamp     Open     High      Low    Close  Volume (BTC)  \
timestamp                                                                    
2017-01-01   1/1/17 0:00   966.34  1005.00   960.53   997.75       6850.59   
2017-01-02   1/2/17 0:00   997.75  1032.00   990.01  1012.54       8167.38   
2017-01-03   1/3/17 0:00  1011.44  1039.00   999.99  1035.24       9089.66   
2017-01-04   1/4/17 0:00  1035.51  1139.89  1028.56  1114.92      21562.46   
2017-01-05   1/5/17 0:00  1114.38  1136.72   885.41  1004.74      36018.86   
...                  ...      ...      ...      ...      ...           ...   
2018-03-24  3/24/18 0:00  8917.99  9020.00  8505.00  8547.00       9731.98   
2018-03-25  3/25/18 0:00  8541.96  8680.00  8368.63  8453.90       9155.91   
2018-03-26  3/26/18 0:00  8451.12  8500.00  7831.15  8149.66      17693.42   
2018-03-27  3/27/18 0:00  8152.26  8211.62  7742.11  7791.70      12385.25   
2018-03-28  3/28/18 0:00  7791.69  8104.98  7723.03  8039.86    

In [9]:
from arctic.date import DateRange

btc_range = DateRange("2017-01-01", "2017-06-30")
btc_df = bitcoin_library.read("BTC", date_range=btc_range).data
print(btc_df)


               Timestamp     Open     High      Low    Close  Volume (BTC)  \
timestamp                                                                    
2017-01-01   1/1/17 0:00   966.34  1005.00   960.53   997.75       6850.59   
2017-01-02   1/2/17 0:00   997.75  1032.00   990.01  1012.54       8167.38   
2017-01-03   1/3/17 0:00  1011.44  1039.00   999.99  1035.24       9089.66   
2017-01-04   1/4/17 0:00  1035.51  1139.89  1028.56  1114.92      21562.46   
2017-01-05   1/5/17 0:00  1114.38  1136.72   885.41  1004.74      36018.86   
...                  ...      ...      ...      ...      ...           ...   
2017-06-26  6/26/17 0:00  2502.26  2550.03  2315.01  2421.22      22094.21   
2017-06-27  6/27/17 0:00  2421.21  2565.00  2291.00  2565.00      19129.60   
2017-06-28  6/28/17 0:00  2565.00  2593.00  2468.01  2559.90      12274.55   
2017-06-29  6/29/17 0:00  2550.99  2594.78  2500.31  2541.59       8125.38   
2017-06-30  6/30/17 0:00  2538.41  2564.58  2451.00  2465.49    

In [11]:
from functools import wraps
import time


def timing_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function {func.__name__} took {end_time - start_time:.2f} seconds")
        return result

    return wrapper


@timing_decorator
def df_csv_read():
    df = pd.read_csv("bitcoin_data.csv")
    return df  # Return the DataFrame


@timing_decorator
def df_arctic_read():
    df = bitcoin_library.read("BTC").data
    return df  # Return the DataFrame


csv_df = df_csv_read()
arctic_df = df_arctic_read()


Function df_csv_read took 0.02 seconds
Function df_arctic_read took 0.01 seconds
